# Set up

In [1]:
library("tidyverse")
library("janitor")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




# Load data

In [2]:
in_csv = "/data/SG10K-DRAGEN/s3/sg10k-dragen/pilot/select_samples/SG10K-DRAGEN.sample_list_full.20210719.csv"
sample_list = read_csv(in_csv, col_types = cols()) %>% 
clean_names

sample_list %>% head
sample_list %>% dim

npm_research_id,multiplex_pool_id,supplier_id,site_supplying_sample,supplied_gender,genetic_sex,self_reported_ethnicity,genetic_ancestry,extraction_kit,library_prep_kit,sequencing_depth,estimate_of_sequence_coverage,qc_pass_in_r5_3,industry_consent
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
FAILED00365,NA,00365,HELIOS,M,NA,Chinese,NA,abGENIX Whole Blood Genomic DNA Extraction Kit,NEBNext UltraII DNA library Prep Kit for Illumina,15x,NA,FALSE,TRUE
FAILED01615,NA,01615,HELIOS,F,NA,Chinese,NA,abGENIX Whole Blood Genomic DNA Extraction Kit,NEBNext UltraII DNA library Prep Kit for Illumina,15x,NA,FALSE,TRUE
FAILED01688,NA,01688,HELIOS,F,NA,Others,NA,abGENIX Whole Blood Genomic DNA Extraction Kit,NEBNext UltraII DNA library Prep Kit for Illumina,15x,NA,FALSE,TRUE
FAILED01750,NA,01750,HELIOS,F,NA,Chinese,NA,abGENIX Whole Blood Genomic DNA Extraction Kit,NEBNext UltraII DNA library Prep Kit for Illumina,15x,NA,FALSE,TRUE
FAILED01799,NA,01799,HELIOS,F,NA,Chinese,NA,abGENIX Whole Blood Genomic DNA Extraction Kit,NEBNext UltraII DNA library Prep Kit for Illumina,15x,NA,FALSE,TRUE
FAILED02755,NA,02755,HELIOS,F,NA,Chinese,NA,abGENIX Whole Blood Genomic DNA Extraction Kit,NEBNext UltraII DNA library Prep Kit for Illumina,15x,NA,FALSE,TRUE


[1] 10714    14

In [3]:
sel_samples = sample_list %>% 
filter(qc_pass_in_r5_3 == TRUE) %>% 
filter(industry_consent == TRUE) %>% 
filter(sequencing_depth == "30x")

In [4]:
sel_samples %>% 
group_by(genetic_ancestry, site_supplying_sample) %>% 
summarise(n = n()) %>% 
spread(genetic_ancestry, n) %>% 
adorn_totals(where = "both")

`summarise()` regrouping output by 'genetic_ancestry' (override with `.groups` argument)



site_supplying_sample,C,I,M,Total
<chr>,<int>,<int>,<int>,<dbl>
GUSTO_Kids,572,161,236,969
SE ASIAN,3,NA,60,63
Total,575,161,296,1032


In [5]:
sel_samples %>% 
group_by(library_prep_kit) %>% 
summarise(n = n())

sel_samples %>% 
filter(library_prep_kit == "Unknown") %>% 
head

`summarise()` ungrouping output (override with `.groups` argument)



library_prep_kit,n
<chr>,<int>
NEBNext UltraII DNA library Prep Kit for Illumina,969
Unknown,63


npm_research_id,multiplex_pool_id,supplier_id,site_supplying_sample,supplied_gender,genetic_sex,self_reported_ethnicity,genetic_ancestry,extraction_kit,library_prep_kit,sequencing_depth,estimate_of_sequence_coverage,qc_pass_in_r5_3,industry_consent
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
SSM002,NA,SSM002,SE ASIAN,F,female,Malay,M,Unknown,Unknown,30x,44.40143,TRUE,TRUE
SSM003,NA,SSM003,SE ASIAN,M,male,Malay,M,Unknown,Unknown,30x,47.47151,TRUE,TRUE
SSM004,NA,SSM004,SE ASIAN,F,female,Malay,M,Unknown,Unknown,30x,51.96134,TRUE,TRUE
SSM007,NA,SSM007,SE ASIAN,F,female,Malay,M,Unknown,Unknown,30x,53.34756,TRUE,TRUE
SSM009,NA,SSM009,SE ASIAN,F,female,Malay,M,Unknown,Unknown,30x,53.65297,TRUE,TRUE
SSM010,NA,SSM010,SE ASIAN,F,female,Malay,M,Unknown,Unknown,30x,42.95423,TRUE,TRUE


In [6]:
out_csv = "/home/jupyter-mgonzalezporta/workspace/data/SG10K-CYP2D6/s3/sg10k-cyp2d6/metadata/SG10K-CYP2D6.sample_list.20210722.csv"
write_csv(sel_samples, file = out_csv)